In [34]:
import numpy as np
import sklearn.metrics as metrics
import pickle

IF=open("assignment3_sql.txt",'r')
lst_code_date=[]
trainX=[]
trainY=[]
for line in IF:
    code, date, x, y = line.strip().split("\t")
    lst_code_date.append([code, date])
    trainX.append([])
    tmpx = list(map(int, x.split(",")))
    for i in range(0,len(tmpx),5):
        trainX[-1].append(tmpx[i:i+5])
    trainY.append(int(y))
trainX=np.array(trainX)
trainY=np.array(trainY)

In [35]:
trainX.shape

(14120, 10, 5)

In [29]:
from tqdm.notebook import tqdm

import tensorflow as tf
from tensorflow import keras

model = keras.models.Sequential()
model.add(keras.layers.InputLayer(input_shape=(trainX.shape[1],trainX.shape[2])))
model.add(keras.layers.LSTM(128))
model.add(keras.layers.LSTM(64, activation='hard_sigmoid', return_sequences=True))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.LSTM(64, activation='hard_sigmoid', return_sequences=True))
model.add(keras.layers.LSTM(32, activation='hard_sigmoid', return_sequences=False))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(1, activation='sigmoid'))
    
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(0.01,decay_steps=100000,decay_rate=0.96)), 
    loss="binary_crossentropy",
    metrics=['accuracy'])
model.summary()

NotImplementedError: Cannot convert a symbolic Tensor (lstm_16/strided_slice:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported

In [30]:
es = keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=20, verbose=1, mode="auto", restore_best_weights=True)
lr = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=10, verbose=1)
sv = keras.callbacks.ModelCheckpoint(
    "./model_keras_lstm.h5", monitor='val_accuracy', verbose=1, save_best_only=True,
    save_weights_only=False, mode='auto', save_freq='epoch',
    options=None
    )

history = model.fit(trainX, trainY, validation_split=0.20, batch_size=128, epochs=50, callbacks=[es, sv], shuffle=True)            

Epoch 1/50
89/89 [==============================] - 1s 13ms/step - loss: 0.5426 - accuracy: 0.7581 - val_loss: 0.5572 - val_accuracy: 0.7585

Epoch 00001: val_accuracy improved from -inf to 0.75850, saving model to ./model_keras_mlp.h5
Epoch 2/50
89/89 [==============================] - 1s 12ms/step - loss: 0.5424 - accuracy: 0.7581 - val_loss: 0.5608 - val_accuracy: 0.7585

Epoch 00002: val_accuracy did not improve from 0.75850
Epoch 3/50
89/89 [==============================] - 1s 10ms/step - loss: 0.5421 - accuracy: 0.7581 - val_loss: 0.5542 - val_accuracy: 0.7585

Epoch 00003: val_accuracy did not improve from 0.75850
Epoch 4/50
89/89 [==============================] - 1s 10ms/step - loss: 0.5419 - accuracy: 0.7581 - val_loss: 0.5602 - val_accuracy: 0.7585

Epoch 00004: val_accuracy did not improve from 0.75850
Epoch 5/50
89/89 [==============================] - 1s 13ms/step - loss: 0.5422 - accuracy: 0.7581 - val_loss: 0.5548 - val_accuracy: 0.7585

Epoch 00005: val_accuracy did n

In [31]:
import numpy as np
import sklearn.metrics as metrics
import pickle
from sklearn import svm
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

IF=open("assignment3-2_sql.txt",'r')
lst_code_date=[]
testX=[]
testY=[]
for line in IF:
    code, date, x, y = line.strip().split("\t")
    lst_code_date.append([code, date])
    testX.append([])
    tmpx = list(map(int, x.split(",")))
    for i in range(0,len(tmpx),5):
        testX[-1].append(tmpx[i:i+5])    
    testY.append(int(y))
testX=np.array(testX)
testY=np.array(testY)

In [32]:
# load a saved model
from keras.models import load_model
saved_model = load_model('./model_keras_lstm.h5')
predY = model.predict(testX)
predY

array([[0.3111323 ],
       [0.3039411 ],
       [0.17294732],
       ...,
       [0.31050873],
       [0.30269128],
       [0.3111323 ]], dtype=float32)

In [40]:
lst_output=[]
for (code, date), y in zip(lst_code_date, predY):
    if y[0] >= 0.31:
        lst_output.append([code, date, "buy", "r10"])
        lst_output.append([code, date+"n", "sell", "all"])

lst_output.sort(key=lambda x:x[1])

OF=open("assignment4.txt",'w')
for row in lst_output:
    OF.write("\t".join(map(str, row))+"\n")
OF.close()